In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cost(output, target):
    return np.mean((output - target) ** 2)

In [2]:
# Layer Class
class Layer:
    def __init__(self, number, size):
        self._number = number
        self._size = size
        self._data = None 

    @property
    def number(self):
        return self._number

    @property
    def size(self):
        return self._size

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_data):
        if new_data.shape[1] == self._size:
            self._data = new_data
        else:
            raise ValueError("Shape mismatch while setting layer data.")

# Weight Class
class Weight:
    def __init__(self, from_layer, to_layer):
        self._value = np.random.randn(from_layer.size, to_layer.size).astype(np.float32)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        if new_value.shape == self._value.shape:
            self._value = new_value
        else:
            raise ValueError("Shape mismatch while setting weight value.")

# Bias Class
class Bias:
    def __init__(self, to_layer):
        self._value = np.random.randn(1, to_layer.size).astype(np.float32)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        if new_value.shape == self._value.shape:
            self._value = new_value
        else:
            raise ValueError("Shape mismatch while setting bias value.")

# Neural Network Class
class Neural:
    def __init__(self, activation):
        self._activation = activation
        self._layers = []
        self._weights = []
        self._biases = []

    def add_layer(self, layer):
        self._layers.append(layer)
        if len(self._layers) > 1:
            self._weights.append(Weight(self._layers[-2], self._layers[-1]))
            self._biases.append(Bias(self._layers[-1]))

    def forward(self):
        for i in range(1, len(self._layers)):
            prev_data = self._layers[i-1].data
            w = self._weights[i-1].value
            b = self._biases[i-1].value
            z = np.dot(prev_data, w) + b
            activated = self._activation(z)
            self._layers[i].data = activated

    @property
    def output(self):
        return self._layers[-1].data


In [3]:
# Load Dataset
file_name = 'mnist_test.csv'
df = pd.read_csv(file_name)

labels = df['label'].values
pixels = df.drop('label', axis=1).values.astype(np.uint8)

# Batch Setup
n = 10
max_idx = len(labels) - 1
idxs = list(range(n))

# Display All Images
for idx in idxs:
    print(f"Label: {labels[idx]}")
    display(Image.fromarray(pixels[idx].reshape(28, 28), mode='L'))

# Normalize and Prepare Input Batch
images = pixels[idxs].astype(np.float32) / 255.0  
targets = np.zeros((n, 10), dtype=np.float32)    
for i in range(n):
    targets[i, labels[idxs[i]]] = 1

# Build Network
net = Neural(sigmoid)
input_layer = Layer(1, 784)
input_layer.data = images
net.add_layer(input_layer)

net.add_layer(Layer(2, 107))
net.add_layer(Layer(3, 26))
net.add_layer(Layer(4, 10))

net.forward()

# Prediction
predictions = np.argmax(net.output, axis=1)
print("Predicted Digits:", predictions)
print("Actual Labels    :", labels[idxs])

# Cost Function
cost_function = cost(net.output, targets)
print("Cost :", cost_function)

Label: 7


Label: 2


Label: 1


Label: 0


Label: 4


...

Label: 4


Label: 9


Label: 5


Label: 9


Predicted Digits: [9 9 9 9 9 9 3 9 9 9]
Actual Labels    : [7 2 1 0 4 1 4 9 5 9]
Cost : 0.39925233
